In [1]:
from pathlib import Path
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from scipy import signal

In [26]:
data_path = Path('../') / 'EEG_data' / 'cleaned_data'
sample = data_path / 'sub-014_cleaned-epo.fif'

sample_2 = data_path / 'sub-0014_cleaned-epo.fif'

In [27]:
cleaned_data_file = sample
epochs = mne.read_epochs(cleaned_data_file, preload=True, verbose=False)
raw_cleaned_df = epochs.to_data_frame()
raw_cleaned_df 

channels = ['C3:A2', 'C4:A1', 'O2:A1', 'F4:A1', 'O1:A2', 'F3:A2']
raw_cleaned_df

,time,condition,epoch,F4:A1,C4:A1,O1:A2,EOG1:A2,EOG2:A1,F3:A2,C3:A2,...,A2,O1,O2,F3,F4,EMG2:EMG3,EMG1:EMG3,F3:C3,F4:C4,EOG2:EOG1
0,0.000000,A,0,2.032879e-14,-2.371692e-14,-1.016440e-14,-6.776264e-15,4.065758e-14,0.000000,1.016440e-14,...,-1.694066e-14,1.016440e-14,8.470329e-15,1.694066e-15,6.776264e-15,6.505213e-13,2.168404e-13,-3.388132e-15,1.694066e-15,-1.355253e-14
1,0.003906,A,0,4.441733e-01,2.577153e-01,1.592687e-01,-2.318195e-01,7.661928e-01,0.343518,1.125843e-01,...,-1.101635e-01,4.910516e-02,9.882910e-03,2.333541e-01,1.298125e-01,-3.189765e+00,-3.115520e+00,2.309334e-01,1.864580e-01,7.938151e-01
2,0.007812,A,0,2.135317e-01,1.315446e-01,1.494793e-01,-2.663979e-01,6.696896e-01,0.303296,1.350191e-01,...,-1.110987e-01,3.838059e-02,1.416554e-02,1.921972e-01,2.560830e-02,2.434808e+00,2.475171e+00,1.682767e-01,8.198708e-02,8.592628e-01
3,0.011719,A,0,-4.681981e-01,-2.521887e-01,2.746998e-02,-1.657902e-01,-4.942624e-04,0.005275,9.843615e-02,...,-4.003734e-02,-1.256737e-02,1.398016e-02,-3.476257e-02,-2.341209e-01,1.364382e+01,1.356818e+01,-9.316140e-02,-2.160094e-01,4.394104e-01
4,0.015625,A,0,-7.774456e-01,-4.254852e-01,4.662695e-04,-1.689165e-01,-1.905620e-01,-0.080840,1.140763e-01,...,-3.125215e-02,-3.078589e-02,1.606755e-02,-1.120922e-01,-3.634639e-01,1.953809e+01,1.939380e+01,-1.949164e-01,-3.519603e-01,4.235883e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11037592,29.984375,Wake,1436,3.691225e-02,2.034163e-01,-1.805490e+00,-1.679599e+00,-6.654116e+00,1.654473,-1.823552e+00,...,-9.613103e-01,-2.766800e+00,-2.401319e+00,6.931630e-01,2.856007e+00,-9.435648e+00,-4.567502e-01,3.478026e+00,-1.665041e-01,-1.194112e+00
11037593,29.988281,Wake,1436,-3.462155e-02,6.925997e-01,-1.257887e+00,-5.061239e-01,-4.989777e+00,1.905301,-9.724975e-01,...,-1.695062e+00,-2.952950e+00,-2.872407e+00,2.102392e-01,2.163551e+00,8.148228e+00,4.979570e+00,2.877799e+00,-7.272214e-01,-5.904184e-01
11037594,29.992188,Wake,1436,4.057083e+00,4.916160e+00,-1.678821e+00,1.086478e+01,6.480693e+00,6.483797,8.237962e+00,...,-6.122637e+00,-7.801457e+00,-8.466347e+00,3.611599e-01,-4.847519e-01,1.506183e+00,-1.135410e+01,-1.754165e+00,-8.590772e-01,-2.803283e+00
11037595,29.996094,Wake,1436,3.728068e+00,4.878546e+00,-1.836834e+00,1.138524e+01,7.532272e+00,6.527276,8.187917e+00,...,-6.639719e+00,-8.476553e+00,-9.147164e+00,-1.124438e-01,-1.042186e+00,-3.143375e+01,-2.934093e+01,-1.660641e+00,-1.150478e+00,-1.983506e+00


FileNotFoundError: File does not exist: "/Users/katieoreilly/Desktop/unsw/SCIF_2001_actual2.nosync/SCIF_2001/notebooks/../EEG_data/cleaned_data/sub-0014_cleaned-epo.fif"

In [4]:
bands = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "sigma": (12, 15),
    "beta": (15, 30),
}

# def get_band_power(fft_freqs, intensity, band):
#     upper = (np.abs(fft_freqs) <= bands['delta'][1])
#     lower = (np.abs(fft_freqs) >= bands['delta'][0])

#     power = intensity[upper & lower].sum()
#     return power

def get_band_power(fft_freqs, psd, band):
    low_freq, high_freq = bands[band]
    freq_res = fft_freqs[1] - fft_freqs[0]
    band_indices = np.where((fft_freqs >= low_freq) & (fft_freqs <= high_freq))[0]
    power = np.sum(psd[band_indices]) * freq_res
    return power 

In [5]:
def feature_extract_fft(raw_cleaned_df, epoch, channel):
    epoch_filt = raw_cleaned_df["epoch"] == epoch
    s = raw_cleaned_df[epoch_filt][channel]
    t = raw_cleaned_df[epoch_filt]["time"]

    label = raw_cleaned_df[epoch_filt]['condition'].unique()[0]

    d = 30 / (len(t) - 1)
    n = len(t)

    fft = 2 * np.fft.fft(s) / n
    fft_freqs = np.fft.fftfreq(n, d)

    intensity = np.abs(fft) ** 2

    peak_freq = np.abs(fft_freqs[np.argmax(intensity)])
    total_power = intensity.sum()
    delta_power = get_band_power(fft_freqs, intensity, "delta")
    theta_power = get_band_power(fft_freqs, intensity, "theta")
    alpha_power = get_band_power(fft_freqs, intensity, "alpha")
    sigma_power = get_band_power(fft_freqs, intensity, "sigma")
    beta_power = get_band_power(fft_freqs, intensity, "beta")

    

    return {
        "peak_freq": peak_freq,
        "total_power": total_power,
        "delta_power": delta_power,
        "theta_power": theta_power,
        "alpha_power": alpha_power,
        "sigma_power": sigma_power,
        "beta_power": beta_power,
    }, label


def feature_extract_welch(raw_cleaned_df, epoch, channel):
    epoch_filt = raw_cleaned_df["epoch"] == epoch
    s = raw_cleaned_df[epoch_filt][channel].to_numpy()
    t = raw_cleaned_df[epoch_filt]["time"]

    fs = 256
    nperseg = 2 * fs
    # The 'welch' function returns frequencies and power spectral density
    fft_freqs, intensity = signal.welch(s, fs=fs, nperseg=nperseg)

    label = raw_cleaned_df[epoch_filt]['condition'].unique()[0]

    peak_freq = np.abs(fft_freqs[np.argmax(intensity)])
    total_power = intensity.sum()
    delta_power = get_band_power(fft_freqs, intensity, "delta")
    theta_power = get_band_power(fft_freqs, intensity, "theta")
    alpha_power = get_band_power(fft_freqs, intensity, "alpha")
    sigma_power = get_band_power(fft_freqs, intensity, "sigma")
    beta_power = get_band_power(fft_freqs, intensity, "beta")

    return {
        "peak_freq": peak_freq,
        "total_power": total_power,
        "delta_power": delta_power,
        "theta_power": theta_power,
        "alpha_power": alpha_power,
        "sigma_power": sigma_power,
        "beta_power": beta_power,
        "delta_relative": delta_power / total_power if total_power > 0 else 0,
        "theta_relative": theta_power / total_power if total_power > 0 else 0,
        "alpha_relative": alpha_power / total_power if total_power > 0 else 0,
        "sigma_relative": sigma_power / total_power if total_power > 0 else 0,
        "beta_relative": beta_power / total_power if total_power > 0 else 0,
    }, label


def feature_extract_welch_only_rel(raw_cleaned_df, epoch, channel):
    epoch_filt = raw_cleaned_df["epoch"] == epoch
    s = raw_cleaned_df[epoch_filt][channel].to_numpy()
    t = raw_cleaned_df[epoch_filt]["time"]

    fs = 256
    nperseg = 2 * fs
    # The 'welch' function returns frequencies and power spectral density
    fft_freqs, intensity = signal.welch(s, fs=fs, nperseg=nperseg)

    label = raw_cleaned_df[epoch_filt]['condition'].unique()[0]

    peak_freq = np.abs(fft_freqs[np.argmax(intensity)])
    total_power = intensity.sum()
    delta_power = get_band_power(fft_freqs, intensity, "delta")
    theta_power = get_band_power(fft_freqs, intensity, "theta")
    alpha_power = get_band_power(fft_freqs, intensity, "alpha")
    sigma_power = get_band_power(fft_freqs, intensity, "sigma")
    beta_power = get_band_power(fft_freqs, intensity, "beta")

    return {
        "peak_freq": peak_freq,
        "total_power": total_power,
        "delta_relative": delta_power / total_power if total_power > 0 else 0,
        "theta_relative": theta_power / total_power if total_power > 0 else 0,
        "alpha_relative": alpha_power / total_power if total_power > 0 else 0,
        "sigma_relative": sigma_power / total_power if total_power > 0 else 0,
        "beta_relative": beta_power / total_power if total_power > 0 else 0,
    }, label


In [6]:
res_df = {}
label_df = {}

# channels_to_keep = [
#         "F4:M1", "C4:M1", 'O1:M2', 
#         'F3:M2', 'C3:M2', 'O2:M1']



# for epoch in raw_cleaned_df["epoch"].unique():
#     res_df[epoch], label_df[epoch] = feature_extract(raw_cleaned_df, epoch, 'C4:A1')

channels_to_keep = ['C3:A2', 'C4:A1', 'O2:A1', 
                    'F4:A1', 'O1:A2', 'F3:A2']

all_epoch_features = [] 

for epoch in raw_cleaned_df["epoch"].unique():
    features_for_this_epoch = {} 
    label = None 
    for ch in channels_to_keep:
        features, label = feature_extract_welch(raw_cleaned_df, epoch, ch)
        for feature_name, value in features.items():
            features_for_this_epoch[f"{ch}_{feature_name}"] = value
    features_for_this_epoch['label'] = label

    all_epoch_features.append(features_for_this_epoch)

final_df = pd.DataFrame(all_epoch_features)



In [7]:
#Split data into

# X = pd.DataFrame(res_df).T
# y = pd.Series(label_df)

X = final_df.drop('label', axis=1)
y = final_df['label']

train_size = int(0.8*len(X))
test_size = len(X) - train_size

shuffle = np.random.choice(np.arange(len(X)), len(X), replace=False)
train_idx = shuffle[0:train_size]
test_idx = shuffle[train_size:]

In [9]:
model = GradientBoostingClassifier()
model.fit(X.iloc[train_idx], y.iloc[train_idx])

train_pred = model.predict(X.iloc[train_idx])
test_pred = model.predict(X.iloc[test_idx])

train_correct = train_pred == y.iloc[train_idx]
test_correct = test_pred == y.iloc[test_idx]

train_accuracy = train_correct.mean()
test_accuracy = test_correct.mean()

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")

Training Accuracy: 1.0000
Testing Accuracy: 0.8328


In [10]:
unique_labels = np.unique(train_pred)
confusion_train = pd.DataFrame(index = unique_labels, columns=unique_labels)
confusion_test = pd.DataFrame(index = unique_labels, columns=unique_labels)

for col in confusion_train.columns:
    for row in confusion_train.index:
        confusion_train.loc[row, col] = (y.iloc[train_idx][train_pred == row] == col).sum()

for col in confusion_test.columns:
    for row in confusion_test.index:
        confusion_test.loc[row, col] = (y.iloc[test_idx][test_pred == row] == col).sum()



In [11]:
confusion_train




,N1,N2,N3,REM,Wake
N1,84,0,0,0,0
N2,0,327,0,0,0
N3,0,0,108,0,0
REM,0,0,0,77,0
Wake,0,0,0,0,552


In [12]:
confusion_test


,N1,N2,N3,REM,Wake
N1,11,0,0,3,3
N2,4,61,9,0,0
N3,0,11,24,0,0
REM,0,0,0,17,0
Wake,11,3,1,2,126


In [287]:
#with one channel 0.59 
#with all on patient 1: 0.55
#with one channel and welch 0.61
#all channels and subject 3  and welch - 0.76
#fft and all channels and subject 3 - 0.663
#welch with all channels and relative pds 0.83
#welch all channel and only relative 0.74 
